# Ollama + OpenAI + Python

## 1. Určite názov modelu

Ak ste použili iný model ako "phi3:mini", zmeňte hodnotu v bunke nižšie. Táto premenná bude použitá v kóde v celom notebooku.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Nastavenie klienta Open AI

Klient OpenAI sa zvyčajne používa s OpenAI.com alebo Azure OpenAI na interakciu s veľkými jazykovými modelmi.  
Môže sa však použiť aj s Ollama, pretože Ollama poskytuje kompatibilný koncový bod OpenAI na adrese "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Vytvorenie odpovede na chat

Teraz môžeme použiť OpenAI SDK na vytvorenie odpovede pre konverzáciu. Táto požiadavka by mala vytvoriť haiku o mačkách:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Návrh výzvy

Prvá správa odoslaná jazykovému modelu sa nazýva "systémová správa" alebo "systémová výzva" a nastavuje celkové pokyny pre model.  
Môžete poskytnúť vlastnú systémovú výzvu, aby ste nasmerovali jazykový model na generovanie výstupu iným spôsobom.  
Upravte `SYSTEM_MESSAGE` nižšie tak, aby odpovedal ako vaša obľúbená postava z filmu alebo televízneho seriálu, alebo získajte inšpiráciu pre ďalšie systémové výzvy z [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Keď upravíte systémovú správu, poskytnite prvú otázku používateľa v `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Príklady s malým počtom ukážok

Ďalším spôsobom, ako usmerniť jazykový model, je poskytnúť "malý počet ukážok", teda sekvenciu príkladov otázok a odpovedí, ktoré ukazujú, ako by mal model reagovať.

Príklad nižšie sa snaží prinútiť jazykový model, aby sa správal ako asistent učiteľa tým, že poskytne niekoľko príkladov otázok a odpovedí, ktoré by mohol asistent učiteľa poskytnúť, a potom modelu položí otázku, ktorú by mohol položiť študent.

Vyskúšajte to najskôr, a potom upravte `SYSTEM_MESSAGE`, `EXAMPLES` a `USER_MESSAGE` pre nový scenár.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Generovanie s podporou vyhľadávania

RAG (Generovanie s podporou vyhľadávania) je technika, ktorá umožňuje jazykovému modelu presne odpovedať na otázky pre konkrétnu oblasť tým, že najskôr vyhľadá relevantné informácie zo zdroja znalostí a následne na základe týchto informácií generuje odpoveď.

Poskytli sme lokálny CSV súbor s údajmi o hybridných autách. Kód nižšie načíta CSV súbor, vyhľadá zhody s otázkou používateľa a potom na základe nájdených informácií vygeneruje odpoveď. Upozorňujeme, že tento proces bude trvať dlhšie ako ktorýkoľvek z predchádzajúcich príkladov, pretože modelu posiela viac údajov. Ak si všimnete, že odpoveď stále nie je založená na údajoch, môžete skúsiť systémové inžinierstvo alebo iné modely. Vo všeobecnosti je RAG efektívnejší buď s väčšími modelmi, alebo s upravenými verziami SLM.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
